# Python Memory Management


This notebook covers **complete memory management in Python**:
- Reference counting
- Circular references
- Garbage collector
- Generational GC
- Memory profiling with `tracemalloc`
- Best practices

Python's memory management is **automatic**, but understanding it helps you write **efficient code**.


## 1. Reference Counting


Python tracks the **number of references** to every object.
When the count becomes **zero**, the object is destroyed immediately.


In [3]:

import sys

x = []  # empty list
print("Initial ref count:", sys.getrefcount(x)-1)

y = x   # Another reference
print("After assigning y:", sys.getrefcount(x)-1)

del y   # Remove a reference
print("After deleting y:", sys.getrefcount(x)-1)


Initial ref count: 1
After assigning y: 2
After deleting y: 1


## 2. Circular References Problem


Reference counting fails when objects **reference each other** in a cycle.


In [ ]:

class Node:
    def __init__(self):
        self.ref = None

a = Node()
b = Node()

a.ref = b
b.ref = a
print("Created circular reference between a and b")


Created circular reference between a and b


## 3. Garbage Collector


Python's **garbage collector** detects and frees objects involved in **circular references**.

It uses **generational garbage collection**:
- **Gen 0**: New objects, collected often
- **Gen 1**: Survived one collection
- **Gen 2**: Survived multiple collections


In [4]:

import gc

print("Is GC enabled?", gc.isenabled())
unreachable = gc.collect()
print(f"Unreachable objects found and collected: {unreachable}")


Is GC enabled? True
Unreachable objects found and collected: 16


### Forcing Garbage Collection

In [5]:

class Node:
    def __init__(self, name):
        self.name = name
        self.ref = None
    def __del__(self):
        print(f"Deleting {self.name}")

a = Node("A")
b = Node("B")

a.ref = b
b.ref = a

del a
del b

print("Collecting garbage...")
gc.collect()


Deleting A
Deleting B


2

## 4. Memory Profiling with `tracemalloc`

In [6]:
# Get the Current and Peak using Memory Profiling

import tracemalloc

tracemalloc.start()

# Allocate memory
data = [i for i in range(100000)]

current, peak = tracemalloc.get_traced_memory()
print(f"Current: {current / 1024:.2f} KB; Peak: {peak / 1024:.2f} KB")

tracemalloc.stop()


Current: 3900.28 KB; Peak: 3911.32 KB


In [7]:
# Get the Current and Peak using Memory Profiling

import tracemalloc

def create_list():
    return [i for i in range(10000)]

def run():
    tracemalloc.start()

    create_list()

    snapshot = tracemalloc.take_snapshot()
    top_stats = snapshot.statistics('lineno')

    print("[ Top 10 ]")
    for stat in top_stats[::]:
        print(stat)

In [8]:
run()

[ Top 10 ]
/usr/lib/python3.11/asyncio/base_events.py:782: size=168 B, count=2, average=84 B
/usr/local/lib/python3.11/dist-packages/tornado/queues.py:248: size=144 B, count=1, average=144 B
/usr/lib/python3.11/asyncio/events.py:84: size=72 B, count=1, average=72 B
/usr/local/lib/python3.11/dist-packages/zmq/sugar/attrsettr.py:45: size=55 B, count=1, average=55 B
/usr/lib/python3.11/selectors.py:468: size=36 B, count=1, average=36 B


## 5. Best Practices


✅ Avoid unnecessary references (`del var_name` when done)  
✅ Break cycles manually (`obj.ref = None`)  
✅ Use context managers (`with open(...) as f`)  
✅ Profile memory usage when optimizing  
✅ Stream data instead of loading huge datasets at once  



## 6. Summary Table

| Term              | Meaning |
|-------------------|---------|
| **Reference Count** | Number of references to an object |
| **Circular Reference** | Objects referencing each other |
| **Garbage Collector** | Finds & frees unreachable objects |
| **Generational GC** | Groups objects into ages for efficiency |
| **tracemalloc** | Tracks memory allocation for debugging |
